In [1]:
import os
import cv2
import time
import numpy as np
import tensorflow as tf

E:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
class Image_set:
    def __init__(self):
        self.num_data = 31461
        self.squared_img_path = ".\\flickr30k_squared"
        self.blured_img_path = ".\\flickr30k_blured"
        self.indicator = 0
        self.index = [i for i in range(31461)]
        self.index = np.random.permutation(self.index)
        
    def next_batch(self,batch_size):
            
        end_indicator = self.indicator + batch_size
        if end_indicator >= self.num_data:
            self.index = np.random.permutation(self.index)
            self.indicator = 0
            end_indicator = self.indicator + batch_size
        assert end_indicator < self.num_data
        
        
        batch_img_index = self.index[self.indicator: end_indicator]
        batch_squared_img = []
        batch_blured_img = []
        
        for img_index in batch_img_index:
            squared_img_path = os.path.join(self.squared_img_path,"%d.jpg" % img_index)
            blured_img_path = os.path.join(self.blured_img_path,"%d.jpg" % img_index)
            
            squared_img = np.array(cv2.imread(squared_img_path,1),dtype = np.float32)
            blured_img = np.array(cv2.imread(blured_img_path,1),dtype = np.float32)
            squared_img = (squared_img-127.5)/127.5
            blured_img = (blured_img-127.5)/127.5
            
            batch_squared_img.append(squared_img)
            batch_blured_img.append(blured_img)
        
        self.indicator = end_indicator
        
        if batch_size ==1:#用于测试
            batch_squared_img = []
            batch_blured_img = []
            squared_img_path = os.path.join(self.squared_img_path,"%d.jpg" % 0)
            blured_img_path = os.path.join(self.blured_img_path,"%d.jpg" % 0)
            squared_img = np.array(cv2.imread(squared_img_path,1),dtype = np.float32)
            blured_img = np.array(cv2.imread(blured_img_path,1),dtype = np.float32)
            squared_img = (squared_img-127.5)/127.5
            blured_img = (blured_img-127.5)/127.5
            batch_squared_img.append(squared_img)
            batch_blured_img.append(blured_img)
        return batch_squared_img,batch_blured_img



In [3]:
x_image = tf.placeholder(tf.float32,[None,256,256,3])
y_image = tf.placeholder(tf.float32,[None,256,256,3])    

conv1_1 = tf.layers.conv2d(x_image,64,(3,3),padding='same',activation=None,name='conv1_1')#256*256*3 -> 256*256*64
conv1_2 = tf.layers.conv2d(conv1_1,64,(3,3),padding='same',activation=None,name='conv1_2')
conv1_3 = tf.layers.conv2d(conv1_2,64,(3,3),padding='same',activation=None,name='conv1_3')
bn1 = tf.nn.relu(tf.layers.batch_normalization(conv1_3))
pooling1 = tf.layers.max_pooling2d(bn1,(2,2),(2,2),name = 'pool1') #256*256*64 -> 128*128*64

conv2_1 = tf.layers.conv2d(pooling1,128,(3,3),padding='same',activation=None,name='conv2_1')#128*128*64 -> 128*128*128
conv2_2 = tf.layers.conv2d(conv2_1,128,(3,3),padding='same',activation=None,name='conv2_2')
conv2_3 = tf.layers.conv2d(conv2_2,128,(3,3),padding='same',activation=None,name='conv2_3')
bn2 = tf.nn.relu(tf.layers.batch_normalization(conv2_3))
pooling2 = tf.layers.max_pooling2d(bn2,(2,2),(2,2),name = 'pool2')#128*128*128 ->64*64*128

conv3_1 = tf.layers.conv2d(pooling2,256,(3,3),padding='same',activation=None,name='conv3_1')#64*64*128 -> 64*64*256
conv3_2 = tf.layers.conv2d(conv3_1,256,(3,3),padding='same',activation=None,name='conv3_2')
conv3_3 = tf.layers.conv2d(conv3_2,256,(3,3),padding='same',activation=None,name='conv3_3')
bn3 = tf.nn.relu(tf.layers.batch_normalization(conv3_3))
pooling3 = tf.layers.max_pooling2d(bn3,(2,2),(2,2),name = 'pool3')#64*64*256 -> 32*32*256

conv4_1 = tf.layers.conv2d(pooling3,512,(3,3),padding='same',activation=None,name='conv4_1')#32*32*256 -> 32*32*512
conv4_2 = tf.layers.conv2d(conv4_1,512,(3,3),padding='same',activation=None,name='conv4_2')
bn4 = tf.nn.relu(tf.layers.batch_normalization(conv4_2))
pooling4 = tf.layers.max_pooling2d(bn4,(2,2),(2,2),name = 'pool4')#32*32*512 -> 16*16*512

conv5_1 = tf.layers.conv2d(pooling4,512,(3,3),padding='same',activation=None,name='conv5_1')#16*16*512 -> 16*16*512
conv5_2 = tf.layers.conv2d(conv5_1,512,(3,3),padding='same',activation=None,name='conv5_2')
bn5 = tf.nn.relu(tf.layers.batch_normalization(conv5_2))
pooling5 = tf.layers.max_pooling2d(bn5,(2,2),(2,2),name = 'pool5')#16*16*512 -> 8*8*512

conv6_1 = tf.layers.conv2d(pooling5,512,(3,3),padding='same',activation=None,name='conv6_1')#8*8*512 -> 8*8*512
conv6_2 = tf.layers.conv2d(conv6_1,512,(3,3),padding='same',activation=None,name='conv6_2')
bn6 = tf.nn.relu(tf.layers.batch_normalization(conv6_2))
pooling6 = tf.layers.max_pooling2d(bn6,(2,2),(2,2),name = 'pool6')#8*8*512 -> 4*4*512

conv7_1 = tf.layers.conv2d(pooling6,512,(3,3),padding='same',activation=None,name='conv7_1')#4*4*512 -> 4*4*512
conv7_2 = tf.layers.conv2d(conv7_1,512,(3,3),padding='same',activation=None,name='conv7_2')
bn7 = tf.nn.relu(tf.layers.batch_normalization(conv7_2))
pooling7 = tf.layers.max_pooling2d(bn7,(2,2),(2,2),name = 'pool7')#4*4*512 -> 2*2*512

conv8_1 = tf.layers.conv2d(pooling7,512,(3,3),padding='same',activation=None,name='conv8_1')#2*2*512 -> 2*2*512
conv8_2 = tf.layers.conv2d(conv8_1,512,(3,3),padding='same',activation=None,name='conv8_2')
bn8 = tf.nn.relu(tf.layers.batch_normalization(conv8_2))
pooling8 = tf.layers.max_pooling2d(bn8,(2,2),(2,2),name = 'pool8')#2*2*512 -> 1*1*512

conv1_transpose = tf.layers.conv2d_transpose(pooling8, 512,[3,3],strides=(2,2),padding='SAME')#1*1*512 -> 2*2*512
conv1_tr_1 = tf.layers.conv2d(conv1_transpose,512,(3,3),padding='same',activation=None,name='conv1_tr_1')
bn1_transpose = tf.nn.relu(tf.layers.batch_normalization(conv1_tr_1))
bn1_transpose_res = bn1_transpose + 0.5*pooling7

conv2_transpose = tf.layers.conv2d_transpose(bn1_transpose_res, 512,[3,3],strides=(2,2),padding='SAME')#2*2*512 -> 4*4*512
conv2_tr_1 = tf.layers.conv2d(conv2_transpose,512,(3,3),padding='same',activation=None,name='conv2_tr_1')
bn2_transpose = tf.nn.relu(tf.layers.batch_normalization(conv2_tr_1))
bn2_transpose_res = bn2_transpose + 0.25*pooling6

conv3_transpose = tf.layers.conv2d_transpose(bn2_transpose_res, 512,[3,3],strides=(2,2),padding='SAME')#4*4*512 -> 8*8*512
conv3_tr_1 = tf.layers.conv2d(conv3_transpose,512,(3,3),padding='same',activation=None,name='conv3_tr_1')
bn3_transpose = tf.nn.relu(tf.layers.batch_normalization(conv3_tr_1))
bn3_transpose_res = bn3_transpose + 0.12*pooling5

conv4_transpose = tf.layers.conv2d_transpose(bn3_transpose_res, 512,[3,3],strides=(2,2),padding='SAME')#8*8*512 -> 16*16*512
conv4_tr_1 = tf.layers.conv2d(conv4_transpose,512,(3,3),padding='same',activation=None,name='conv4_tr_1')
bn4_transpose = tf.nn.relu(tf.layers.batch_normalization(conv4_tr_1))
bn4_transpose_res = bn4_transpose + 0.06*pooling4

conv5_transpose = tf.layers.conv2d_transpose(bn4_transpose_res, 256,[3,3],strides=(2,2),padding='SAME')#16*16*512->32*32*256
conv5_tr_1 = tf.layers.conv2d(conv5_transpose,256,(3,3),padding='same',activation=None,name='conv5_tr_1')
conv5_tr_2 = tf.layers.conv2d(conv5_tr_1,256,(3,3),padding='same',activation=None,name='conv5_tr_2')
bn5_transpose = tf.nn.relu(tf.layers.batch_normalization(conv5_tr_2))
bn5_transpose_res = bn5_transpose + 0.03*pooling3

conv6_transpose = tf.layers.conv2d_transpose(bn5_transpose_res, 128,[3,3],strides=(2,2),padding='SAME')#32*32*256->64*64*128
conv6_tr_1 = tf.layers.conv2d(conv6_transpose,128,(3,3),padding='same',activation=None,name='conv6_tr_1')
conv6_tr_2 = tf.layers.conv2d(conv6_tr_1,128,(3,3),padding='same',activation=None,name='conv6_tr_2')
bn6_transpose = tf.nn.relu(tf.layers.batch_normalization(conv6_tr_2))
bn6_transpose_res = bn6_transpose + 0.015*pooling2

conv7_transpose = tf.layers.conv2d_transpose(bn6_transpose_res, 64,[3,3],strides=(2,2),padding='SAME')#64*64*128->128*128*64
conv7_tr_1 = tf.layers.conv2d(conv7_transpose,64,(3,3),padding='same',activation=None,name='conv7_tr_1')
conv7_tr_2 = tf.layers.conv2d(conv7_tr_1,64,(3,3),padding='same',activation=None,name='conv7_tr_2')
bn7_transpose = tf.nn.relu(tf.layers.batch_normalization(conv7_tr_1))
bn7_transpose_res = bn7_transpose #+ 0.0000*pooling1

y_image_predict = tf.nn.relu(tf.layers.conv2d_transpose(bn7_transpose_res, 3,[3,3],strides=(2,2),padding='SAME'))#128*128*64->256*256*3

loss = tf.reduce_mean(tf.square(y_image-y_image_predict))
tf.summary.scalar('loss',loss)
global_step = tf.Variable(0,trainable=False)#训练的次数

In [4]:
batch_size = 32
learning_rate = 0.001
train_steps = 100000
img_set = Image_set()


model_dir = '.\\run_model'
log_dir ='.\\run_log'
test_dir = '.\\test_img'
saver = tf.train.Saver()


with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss,global_step=global_step)
init_op = tf.global_variables_initializer()

with tf.Session() as sess:
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter(log_dir,sess.graph)
    sess.run(init_op)
    ckpt = tf.train.get_checkpoint_state(model_dir)
    if ckpt and ckpt.model_checkpoint_path:
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        saver.restore(sess, os.path.join(model_dir, ckpt_name))
        
    test  = True    
    for step in range(train_steps):
    #for step in range(1):
        if not test:
            start_time = time.time()
            batch_squared_img,batch_blured_img = img_set.next_batch(batch_size)
            global_step_val,losses,losses_merged,_= sess.run([global_step,loss,merged,train_op],
                                                          feed_dict={x_image:batch_blured_img , y_image:batch_squared_img})
        
            use_time = time.time()-start_time
            print("global_step=",global_step_val," losses=",losses," time=%02fs"%use_time)
        
            if global_step_val%20 ==0:
                writer.add_summary(losses_merged,global_step_val)
                saver.save(sess,os.path.join(model_dir,'ckp-%06d' %global_step_val))
                print('model saved to ckp-%06d' %global_step_val)
                test = True
            
        else:    #每20步测试一下图片生成情况
            batch_squared_img,batch_blured_img = img_set.next_batch(1)
            global_step_val,generated_imgs = sess.run([global_step,y_image_predict],
                                                        feed_dict={x_image:batch_blured_img , y_image:batch_squared_img})
            #不训练执行train_op的时候，global_step_val不会+1
            img = generated_imgs[-1]
            img = (img+1)*127.5
            img = np.array(img,dtype = np.uint8)
            out_put_path = os.path.join(test_dir,"%06d.jpg" %global_step_val)
            cv2.imwrite(out_put_path,img)
            #cv2.imshow('a',img)
            #cv2.waitKey()
            test = False
            
            
        

            
            

INFO:tensorflow:Restoring parameters from .\run_model\ckp-000080
global_step= 81  losses= 0.3479114  time=111.867347s
global_step= 82  losses= 223.26933  time=101.384331s
global_step= 83  losses= 0.37984633  time=98.641225s
global_step= 84  losses= 0.75691444  time=97.887517s
global_step= 85  losses= 0.3532084  time=98.976274s
global_step= 86  losses= 0.41487536  time=99.548410s
global_step= 87  losses= 0.48955622  time=98.842290s
global_step= 88  losses= 0.34246767  time=101.156269s
global_step= 89  losses= 0.29893652  time=100.512218s
global_step= 90  losses= 0.36251378  time=99.130208s
global_step= 91  losses= 0.33746898  time=100.886712s
global_step= 92  losses= 0.33681068  time=99.163257s
global_step= 93  losses= 0.3242388  time=98.563246s
global_step= 94  losses= 0.31400704  time=98.882285s
global_step= 95  losses= 0.37026867  time=98.260245s
global_step= 96  losses= 0.32392922  time=98.515222s
global_step= 97  losses= 0.31775248  time=98.191720s
global_step= 98  losses= 0.368459

global_step= 229  losses= 0.34740353  time=101.336297s
global_step= 230  losses= 0.34332335  time=100.607281s
global_step= 231  losses= 0.34420648  time=99.642288s
global_step= 232  losses= 0.3194202  time=99.046299s
global_step= 233  losses= 0.3248087  time=99.099277s
global_step= 234  losses= 0.34727302  time=99.099275s
global_step= 235  losses= 0.33357897  time=100.095259s
global_step= 236  losses= 0.3604271  time=99.717245s
global_step= 237  losses= 0.33442697  time=100.583212s
global_step= 238  losses= 0.32315525  time=100.581251s
global_step= 239  losses= 0.3731998  time=99.784267s
global_step= 240  losses= 0.32954165  time=99.196308s
model saved to ckp-000240
global_step= 241  losses= 0.32434973  time=102.891284s
global_step= 242  losses= 0.36348686  time=101.139308s
global_step= 243  losses= 0.3132596  time=99.864247s
global_step= 244  losses= 0.33833444  time=98.826214s
global_step= 245  losses= 0.35432807  time=99.240271s
global_step= 246  losses= 0.32482964  time=100.875293s

KeyboardInterrupt: 